In [1]:
import cv2
import numpy  as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def display(window_name, canvas):
    cv2.imshow(window_name, canvas)
    cv2.waitKey(0)
    cv2.destroyWindow(window_name)

In [3]:
#defining colors
white = (255,255,255)
black = (0,0,0)
green = (0,255,0)
blue = (255, 0,0)
red = (0,0,255)

# Convolutions with OpenCV

<details>
  <summary> ⭐⭐⭐⭐ OpenCV Image Convolutions (CLICK) </summary>


<p>
In reality, an (image) convolution is simply an element-wise multiplication of two matrices followed by a sum. <br>
    1 . Take two matrices (which both have the same dimensions). <br>
    2 . Multiply them, element-by-element (i.e., not the dot-product, just a simple multiplication). <br>
    3 . Sum the elements together. <br> <br>
    Think of an image as a big matrix and kernel or convolutional matrix as a tiny matrix that is used for blurring, sharpening, edge detection, and other image processing functions. This tiny kernel sits on top of the big image and slides from left-to-right and top-to-bottom, applying a mathematical operation (i.e., a convolution) at each (x, y)-coordinate of the original image. <br> <br>
    Kernels can be an arbitrary size of M x N pixels, provided that both M and N are odd integers. <br>
    1. Select an (x, y)-coordinate from the original image. <br>
    2. Place the center of the kernel at this (x, y)-coordinate. <br>
    3. Take the element-wise multiplication of the input image region and the kernel, then sum up the values of these multiplication operations into a single value. The sum of these multiplications is called the kernel output. <br>
    4. Use the same (x, y)-coordinates from Step #1, but this time, store the kernel output in the same (x, y)-location as the output image. <br>
</p>
  
</details>



In [4]:
import skimage